In [ ]:
# Code used is currently heavily based on a tutorial getting started with variational autoencoder using pytorch on Debugger Cafe
# This will be changed over time

In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader

import torch.nn.functional as F

In [2]:
import os
import pandas as pd
import torch

from torch.utils.data import Dataset

from skimage import io

class ActiveVisionDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, index_col=None)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
            index = index.item()
        #Maybe this should be IMG?
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))
        #cam_loc = torch.tensor(int(self.annotations.iloc[index,2]))
        
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label)

In [3]:
os.chdir("imgs")

In [4]:
dataset = ActiveVisionDataset(csv_file='rgbCSV.csv', root_dir= 'rgbImg/', transform = torchvision.transforms.ToTensor())

train_data, val_data = torch.utils.data.random_split(dataset, [80, 20])
train_loader = DataLoader(dataset=train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=4, shuffle=True)

In [5]:
#for data in train_loader:
#    print(im_path)
#    print("Data: ", data)

In [6]:
print(list(train_data))

[(tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.8627, 0.8549, 0.8706,  ..., 0.6353, 0.6353, 0.6353],
         [0.8588, 0.8588, 0.8627,  ..., 0.6353, 0.6353, 0.6353],
         [0.8588, 0.8667, 0.8667,  ..., 0.6353, 0.6353, 0.6353]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.7647, 0.7529, 0.7608,  ..., 0.7294, 0.7294, 0.7294],
         [0.7529, 0.7608, 0.7647,  ..., 0.7294, 0.7294, 0.7294],
         [0.7529, 0.7686, 0.7686,  ..., 0.7294, 0.7294, 0.7294]],

        [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 

In [7]:
features = 16
# define a simple linear VAE
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
 
        # encoder
        self.enc1 = nn.Linear(in_features=150528, out_features=512)
        self.enc2 = nn.Linear(in_features=512, out_features=features*2)
 
        # decoder 
        self.dec1 = nn.Linear(in_features=features, out_features=512)
        self.dec2 = nn.Linear(in_features=512, out_features=150528)
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x).view(-1, 2, features)
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = torch.sigmoid(self.dec2(x))
        return reconstruction, mu, log_var

In [8]:
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
#import model
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
matplotlib.style.use('ggplot')

In [9]:
# leanring parameters
epochs = 100
batch_size = 4
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
model = LinearVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss(reduction='sum')

In [11]:
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [12]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        data, _ = data
        data = data.to(device)
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [13]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, _ = data
            data = data.to(device)
            data = data.view(data.size(0), -1)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(val_data)/dataloader.batch_size) - 1:
                num_rows = 8
                both = torch.cat((data.view(batch_size, 3, 224, 224)[:8], 
                                  reconstruction.view(batch_size, 3, 224, 224)[:8]))
                save_image(both.cpu(), f"../outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [14]:
print(model)

LinearVAE(
  (enc1): Linear(in_features=150528, out_features=512, bias=True)
  (enc2): Linear(in_features=512, out_features=32, bias=True)
  (dec1): Linear(in_features=16, out_features=512, bias=True)
  (dec2): Linear(in_features=512, out_features=150528, bias=True)
)


In [15]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = fit(model, train_loader)
    val_epoch_loss = validate(model, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Epoch 1 of 100


 15%|████████████▍                                                                      | 3/20 [00:00<00:00, 29.83it/s]

Train Loss: 104717.9277
Val Loss: 94935.0141
Epoch 2 of 100


 15%|████████████▍                                                                      | 3/20 [00:00<00:00, 28.82it/s]

...

Train Loss: 69956.2422
Val Loss: 63971.4547
Epoch 99 of 100


 15%|████████████▍                                                                      | 3/20 [00:00<00:00, 28.28it/s]

Train Loss: 69933.0111
Val Loss: 64077.7281
Epoch 100 of 100


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 39.34it/s]

Train Loss: 69978.6277
Val Loss: 64066.3156


## Random stuff at the bottom

In [ ]:
import os
import pandas as pd
import torch

os.chdir("imgs")

In [ ]:
from torch.utils.data import Dataset

from skimage import io

root_dir= 'rgbImg/'
csv_file='rgbCSV.csv'

annotations = pd.read_csv('rgbCSV.csv', index_col=None)

img_path = os.path.join(root_dir, annotations.iloc[0, 0])
image = io.imread(img_path)
io.imshow(image)
image.shape

In [ ]:
dataset = ActiveVisionDataset(csv_file='rgbCSV.csv', root_dir= 'rgbImg/', transform = torchvision.transforms.ToTensor())

train_set, val_set = torch.utils.data.random_split(dataset, [80, 20])
train_loader = DataLoader(dataset=train_set, batch_size=4, shuffle=True)
val_loader = DataLoader(dataset=valt, batch_size=4, shuffle=True)